In [4]:
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import string
from nltk.corpus import stopwords  # Importa la librería stopwords de NLTK

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
# Cargar el archivo JSONL
with open('meta_Digital_Music.jsonl', 'r') as f:
    data = [json.loads(line) for line in f]

In [7]:
data_df_meta = pd.DataFrame(data)
data_df_meta.head(10)

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together
0,Digital Music,Baja Marimba Band,4.9,8,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],None,[],"{'Date First Available': 'February 28, 2010'}",B000V87RP2,None
1,Digital Music,'80s Halloween-All Original Artists & Recordings,5.0,3,[],[],14.98,[{'thumb': 'https://m.media-amazon.com/images/...,[],"Love and Rockets (Artist), Duran Duran (...",[],{'Package Dimensions': '5.55 x 4.97 x 0.54 inc...,B0062F0MJQ,None
2,Digital Music,TRIO +1,5.0,1,[],[CD ALBUM],57.99,[{'thumb': 'https://m.media-amazon.com/images/...,[],Rob Wasserman Format: Audio CD,[],"{'Is Discontinued By Manufacturer': 'No', 'Pac...",B00005GT12,None
3,Digital Music,"Gold and Silver: Lehar, Delibes, Lanner, Johan...",5.0,1,[],[],29.91,[{'thumb': 'https://m.media-amazon.com/images/...,[],"Franz Lehar (Composer), Leo Delibes (Com...",[],"{'Manufacturer': 'Hungaroton / White Label', '...",B0007PD2BW,None
4,Digital Music,Grateful Dead Dave's Picks Volume 25 Live at B...,4.9,20,[],[Sold out. Numbered limited edition],149.99,[{'thumb': 'https://m.media-amazon.com/images/...,[],"Grateful Dead (Artist, Orchestra) Format: ...",[],{'Package Dimensions': '5.55 x 4.97 x 0.54 inc...,B079CPD45R,None
5,Digital Music,Guitar Escape - Music For Relaxation,4.1,18,[],"[2-disc CD set. Disc 1 ""Escape"" by Steve Middl...",NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],"Guitar Escape (Artist), Steve Middleton ...",[],{'Package Dimensions': '5.55 x 4.97 x 0.54 inc...,B00NFJBVIS,None
6,Digital Music,CLASICOS DE ORO,5.0,1,[],[],20.98,[{'thumb': 'https://m.media-amazon.com/images/...,[],Los Cardenales del Exito (Artist) Format: ...,[],"{'Date First Available': 'February 27, 2010'}",B001PH5SX6,None
7,Digital Music,Indigo for Quantum Focus (Metamusic),5.0,1,[],[Metamusic is more than music.It incorporates ...,28.99,[{'thumb': 'https://m.media-amazon.com/images/...,[],Hemi Sync Format: Audio CD,[],{'Package Dimensions': '5.6 x 4.8 x 0.1 inches...,1561023280,None
8,Digital Music,Fastway,4.7,754,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Fastway,[],"{'Date First Available': 'April 13, 2020'}",B0872KL2TZ,None
9,Digital Music,The Minimalism of Erik Satie,4.8,8,[],[The Minimalim of Erik Satie by The Vienna Art...,9.86,[{'thumb': 'https://m.media-amazon.com/images/...,[],Vienna Art Orchestra Format: Audio CD,[],"{'Is Discontinued By Manufacturer': 'No', 'Ite...",B000025YVO,None


In [8]:
# Cargar el archivo JSONL
with open('Digital_Music.jsonl', 'r') as f:
    data = [json.loads(line) for line in f]

In [9]:
data_df = pd.DataFrame(data)
data_df.head(10)

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5.0,Nice,If i had a dollar for how many times I have pl...,[],B004RQ2IRG,B004RQ2IRG,AFUOYIZBU3MTBOLYKOJE5Z35MBDA,1618972613292,0,True
1,5.0,Excellent,awesome sound - cant wait to see them in perso...,[],B0026UZEI0,B0026UZEI0,AHGAOIZVODNHYMNCBV4DECZH42UQ,1308167525000,0,True
2,5.0,Great service,This is a great cd. Good music and plays well....,[],B0055JSYHC,B0055JSYHC,AFGEM6BXCYHUILEOA3P2ZYBEF2TA,1615838793006,0,True
3,1.0,No good,"These are not real German singers, they have a...",[],B000F9SMUQ,B000F9SMUQ,AH3OG6QD6EDJGZRVCFKV4B66VWNQ,1405219741000,0,True
4,3.0,"Cool concept, so-so execution...",I first heard this playing in a Nagoya shop an...,[],B0049D1WVK,B0049D1WVK,AFW2PDT3AMT4X3PYQG7FJZH5FXFA,1309029595000,0,False
5,4.0,"""L'attaque des clones"" a satisfying taste of t...","I just saw ""Star Wars: L'attaque des clones"" l...",[],B00003CXKT,B00003CXKT,AFW2PDT3AMT4X3PYQG7FJZH5FXFA,1022191122000,1,False
6,5.0,Ronnie Dunn like I've never heard before!,"This is without a doubt my favorite CD, and I ...",[],B083QN4Y6H,B083QN4Y6H,AGYEAZK4OEYF2MSSTGJ5WNJDVZKA,1592615999887,0,True
7,5.0,A true classic seventies rock album,Sounds great,[],B08L7CJ6NM,B08L7CJ6NM,AHB5CGLYN3Y6NIPHNQLYFJT2W2PQ,1641330592178,0,True
8,5.0,A great collection of music,This disk is fantastic I wanted the record but...,[],B092C24W4G,B092C24W4G,AHB5CGLYN3Y6NIPHNQLYFJT2W2PQ,1641224888411,2,True
9,5.0,Excellent musical experience,This is a fantastic collection of music superb...,[],B01M32UHXG,B01M32UHXG,AHB5CGLYN3Y6NIPHNQLYFJT2W2PQ,1566575806068,0,False


In [10]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130434 entries, 0 to 130433
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   rating             130434 non-null  float64
 1   title              130434 non-null  object 
 2   text               130434 non-null  object 
 3   images             130434 non-null  object 
 4   asin               130434 non-null  object 
 5   parent_asin        130434 non-null  object 
 6   user_id            130434 non-null  object 
 7   timestamp          130434 non-null  int64  
 8   helpful_vote       130434 non-null  int64  
 9   verified_purchase  130434 non-null  bool   
dtypes: bool(1), float64(1), int64(2), object(6)
memory usage: 9.1+ MB


In [11]:
filtered_df = data_df[data_df['verified_purchase'] == True]
selected_data = filtered_df[['user_id', 'rating', 'text', 'parent_asin']]

In [12]:
selected_data.head(20)

,user_id,rating,text,parent_asin
0,AFUOYIZBU3MTBOLYKOJE5Z35MBDA,5.0,If i had a dollar for how many times I have pl...,B004RQ2IRG
1,AHGAOIZVODNHYMNCBV4DECZH42UQ,5.0,awesome sound - cant wait to see them in perso...,B0026UZEI0
2,AFGEM6BXCYHUILEOA3P2ZYBEF2TA,5.0,This is a great cd. Good music and plays well....,B0055JSYHC
3,AH3OG6QD6EDJGZRVCFKV4B66VWNQ,1.0,"These are not real German singers, they have a...",B000F9SMUQ
6,AGYEAZK4OEYF2MSSTGJ5WNJDVZKA,5.0,"This is without a doubt my favorite CD, and I ...",B083QN4Y6H
7,AHB5CGLYN3Y6NIPHNQLYFJT2W2PQ,5.0,Sounds great,B08L7CJ6NM
8,AHB5CGLYN3Y6NIPHNQLYFJT2W2PQ,5.0,This disk is fantastic I wanted the record but...,B092C24W4G
11,AHB5CGLYN3Y6NIPHNQLYFJT2W2PQ,5.0,It exceeds my expectations I wish all retailer...,B003NXEPY8
12,AFUB7CHTXRPD447QVQCHBZVN2IPQ,5.0,"astoundingly beautiful, haunting, moving Chora...",B0012N3PMY
13,AFGWR7GZLOWSTFN2QJBE5NMGVEGQ,5.0,Fabulous Cd well worth purchasing...,B006X08FR6


In [13]:
def clean_reviews(reviews):
  """Limpia las reseñas de texto eliminando caracteres especiales, convirtiéndolas a minúsculas y eliminando palabras vacías.

  Args:
      reviews: Una lista de reseñas de texto.

  Returns:
      Una lista de reseñas limpiadas.
  """

  clean_reviews = []
  stop_words = stopwords.words('english')  # Descarga e inicializa las stop words en inglés

  for review in reviews:
    # Convertir a minúsculas
    review_lower = review.lower()

    # Eliminar caracteres de puntuación
    review_sin_puntuacion = ''.join(c for c in review_lower if c not in string.punctuation)

    # Tokenizar el texto (convertir en una lista de palabras)
    review_tokens = review_sin_puntuacion.split()

    # Eliminar stop words
    review_clean = [palabra for palabra in review_tokens if palabra not in stop_words]

    # Unir las palabras limpias en una cadena de texto
    review_clean_str = ' '.join(review_clean)

    clean_reviews.append(review_clean_str)

  return clean_reviews

In [14]:
# Limpiar columna numérica (eliminar NaN)
def clean_numeric_data(data):
    return data.dropna()

In [15]:
# Aplicar la función de limpieza en base al tipo de dato
clean_df = selected_data.copy()

In [16]:
clean_df = clean_numeric_data(clean_df)

In [17]:
clean_df['text'] = clean_reviews(clean_df['text'].tolist())

In [18]:
clean_df.head(20)

,user_id,rating,text,parent_asin
0,AFUOYIZBU3MTBOLYKOJE5Z35MBDA,5.0,dollar many times played cd many times asked a...,B004RQ2IRG
1,AHGAOIZVODNHYMNCBV4DECZH42UQ,5.0,awesome sound cant wait see person always miss...,B0026UZEI0
2,AFGEM6BXCYHUILEOA3P2ZYBEF2TA,5.0,great cd good music plays well seller responde...,B0055JSYHC
3,AH3OG6QD6EDJGZRVCFKV4B66VWNQ,1.0,real german singers accents nothing advertised...,B000F9SMUQ
6,AGYEAZK4OEYF2MSSTGJ5WNJDVZKA,5.0,without doubt favorite cd may close wearing en...,B083QN4Y6H
7,AHB5CGLYN3Y6NIPHNQLYFJT2W2PQ,5.0,sounds great,B08L7CJ6NM
8,AHB5CGLYN3Y6NIPHNQLYFJT2W2PQ,5.0,disk fantastic wanted record recieved couple b...,B092C24W4G
11,AHB5CGLYN3Y6NIPHNQLYFJT2W2PQ,5.0,exceeds expectations wish retailers great mayan,B003NXEPY8
12,AFUB7CHTXRPD447QVQCHBZVN2IPQ,5.0,astoundingly beautiful haunting moving chorale,B0012N3PMY
13,AFGWR7GZLOWSTFN2QJBE5NMGVEGQ,5.0,fabulous cd well worth purchasing,B006X08FR6


In [19]:
processing_data = clean_df.copy()

In [85]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, Concatenate, Dense, Flatten, Dropout
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model

In [21]:
# Parámetros
max_length = 500  # Longitud máxima de las secuencias de texto
vocab_size = 10000  # Tamaño del vocabulario
embedding_dim = 128  # Dimensión del vector de embedding

In [22]:
# Convertir las columnas categóricas a valores numéricos
user_encoder = LabelEncoder()
processing_data['user_id'] = user_encoder.fit_transform(processing_data['user_id'])

song_encoder = LabelEncoder()
processing_data['parent_asin'] = song_encoder.fit_transform(processing_data['parent_asin'])

In [23]:
# Tokenización del texto
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(processing_data['text'])
text_sequences = tokenizer.texts_to_sequences(processing_data['text'])
text_padded = pad_sequences(text_sequences, maxlen=max_length)

In [86]:
# Creación del modelo de red neuronal
user_input = Input(shape=(1,), dtype=tf.int32, name='user_input')
text_input = Input(shape=(max_length,), dtype=tf.int32, name='text_input')
song_input = Input(shape=(1,), dtype=tf.int32, name='song_input')

user_embedding = Embedding(input_dim=len(user_encoder.classes_), output_dim=embedding_dim, name='user_embedding')(user_input)
song_embedding = Embedding(input_dim=len(song_encoder.classes_), output_dim=embedding_dim, name='song_embedding')(song_input)
text_embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, name='text_embedding')(text_input)

user_vecs = Flatten()(user_embedding)
song_vecs = Flatten()(song_embedding)
text_vecs = Flatten()(text_embedding)

concatenated_embeddings = Concatenate()([user_vecs, song_vecs, text_vecs])

dense1 = Dense(64, activation='relu')(concatenated_embeddings)
dense1 = Dropout(0.5)(dense1)
dense2 = Dense(32, activation='relu')(dense1)
dense2 = Dropout(0.5)(dense2)
output = Dense(1)(dense2)

model = tf.keras.Model(inputs=[user_input, text_input, song_input], outputs=output)

In [87]:
# Compilación del modelo
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

In [89]:
# Entrenamiento del modelo
model.fit([processing_data['user_id'], text_padded, processing_data['parent_asin']], processing_data['rating'], epochs=10, validation_split=0.2)

In [ ]:
# Guardar el modelo
model.save('music_recommendation_model.h5')
print("Modelo guardado en 'music_recommendation_model.h5'")

In [24]:
model = load_model('music_recommendation_model.h5')

In [25]:
# Evaluación del modelo
model.evaluate([processing_data['user_id'], text_padded, processing_data['parent_asin']], processing_data['rating'])

3002/3002 [==============================] - 45s 15ms/step - loss: 0.1452 - mae: 0.1451


[0.1452028900384903, 0.1451386660337448]

In [26]:
# Función para predecir la calificación de una canción
def predict_rating(song):
    song_encoded = np.array([song])
    user_array = np.array([user_encoded])
    user_text_array = np.array([user_text_padded[0]])
    predicted_rating = model.predict([user_array, user_text_array, song_encoded])
    return (song, predicted_rating[0][0])

In [27]:
# Generación de recomendaciones para un usuario específico
user_encoded = processing_data['user_id'].sample(1).values[0]

# Preprocesamiento del texto para el usuario seleccionado
user_text = processing_data[processing_data['user_id'] == user_encoded]['text'].values[0]
user_text_sequence = tokenizer.texts_to_sequences([user_text])
user_text_padded = pad_sequences(user_text_sequence, maxlen=max_length)

# Predecir calificaciones para todas las canciones usando map
all_songs = processing_data['parent_asin'].unique()
predicted_ratings = list(map(predict_rating, all_songs))

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 41ms/step


In [39]:
sorted_predicted_song_names = [rec[0] for rec in predicted_ratings]

In [42]:
# Decodificar los IDs de las canciones a nombres originales
song_names = song_encoder.inverse_transform(sorted_predicted_song_names)

In [46]:
# Ordenar las predicciones por calificación descendente
predicted_ratings.sort(key=lambda x: x[1], reverse=True)

# Obtener las 10 mejores recomendaciones
top_10_recommendations = predicted_ratings[:10]

# Decodificar los IDs de las canciones a nombres originales
top_10_song_names = song_encoder.inverse_transform([rec[0] for rec in top_10_recommendations])

print(f"Top 10 canciones recomendadas para el usuario {user_encoder.inverse_transform([user_encoded])}: {top_10_song_names}")

Top 10 canciones recomendadas para el usuario ['AHYNP3TBXZ2AN7DOI36NX7QYFDPA']: ['B09GLPHKDD' 'B000046RW5' 'B006XZWAWM' 'B00AW0M40C' 'B08R6F7Z27'
 'B00OIL3UYA' '3785757387' 'B07YLCHF97' 'B00LVANG9K' 'B000EYDHXO']


In [84]:
recommended_top10_songs = []
for song in top_10_song_names:
  recommended_top10_songs.extend(data_df_meta[data_df_meta['parent_asin'].isin([song])]['title'].values)

print(f"Top 10 canciones recomendadas para el usuario {user_encoder.inverse_transform([user_encoded])}: {recommended_top10_songs}")
for i, song_name in enumerate(recommended_top10_songs, start=1):
  print(f'Canción recomendada {i}: {song_name} \n')

Top 10 canciones recomendadas para el usuario ['AHYNP3TBXZ2AN7DOI36NX7QYFDPA']: ['Meinaoer Water Hyacinth Wicker Storage Baskets, Hand-Woven Cube Storage Baskets for Shelves,Living Room, Bathroom, Bedroom, 12x12x12 inches, 6 Pack', 'Lola-Cola', "My Heart Will Go on - Soothing Guitar Magic. The World's Most Beautiful Melodies By Reader's Digest.", "Ho'oponopono, the Hawiian healing miracle with a Hypnosis Twist. From Wendi", "TWICE - 2021 Season's Greetings, The Moment Forever incl. Making DVD, Keyring, Out Box, AR Photocard Set, Standing Calendar, Paper Perfume, Diary, Mini Message Book, Extra Photocards", 'キノ・アルバム - 会いたい (スマートミュージックカード)(韓国盤)', 'Classics,Folge 38: die Totenkopf-Gang', 'Back To Black - Exclusive Limited Edition White Colored Vinyl LP [Condition-VG+NM]', 'Ohne Stecker', 'Clairvoyance, Psychic & Spiritual Learning Guided Meditation Hypnosis']
Canción recomendada 1: Meinaoer Water Hyacinth Wicker Storage Baskets, Hand-Woven Cube Storage Baskets for Shelves,Living Room, Bat